Demonstrations for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.core_expr_types.lambda_maps</a>
========

In [ ]:
import proveit
from proveit import Lambda, Conditional
from proveit import x, fx, gx, Qx
from proveit.logic import Equals, Forall
%begin demonstrations

### Lambda substitution

In [ ]:
universal_eq = Forall(x, Equals(fx, gx))

In [ ]:
Lambda(x, fx).substitution(universal_eq, assumptions=[universal_eq])

In [ ]:
universal_eq_with_cond = Forall(x, Equals(fx, gx), condition=Qx)

In [ ]:
Lambda(x, Conditional(fx, Qx)).substitution(universal_eq_with_cond, 
                                            assumptions=[universal_eq_with_cond])

In [ ]:
%end demonstrations